
# HW6 Diffusion Model

**Sources:**
- Github implementation [Denoising Diffusion Pytorch](https://github.com/lucidrains/denoising-diffusion-pytorch)
- Papers on Diffusion models ([Dhariwal, Nichol, 2021], [Ho et al., 2020] ect.)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Packages and Set Seeds

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/datasets/LeoFeng/MLHW_6
!unzip ./MLHW_6/faces.zip -d .
# install required dependencies
!pip install einops
!pip install transformers
!pip install ema_pytorch
!pip install accelerate

串流輸出內容已截斷至最後 5000 行。
  inflating: ./faces/14505.jpg       
  inflating: ./faces/42079.jpg       
  inflating: ./faces/41465.jpg       
  inflating: ./faces/32789.jpg       
  inflating: ./faces/57863.jpg       
  inflating: ./faces/23823.jpg       
  inflating: ./faces/62763.jpg       
  inflating: ./faces/28796.jpg       
  inflating: ./faces/46610.jpg       
  inflating: ./faces/26265.jpg       
  inflating: ./faces/9561.jpg        
  inflating: ./faces/65514.jpg       
  inflating: ./faces/54131.jpg       
  inflating: ./faces/66340.jpg       
  inflating: ./faces/52297.jpg       
  inflating: ./faces/63935.jpg       
  inflating: ./faces/48849.jpg       
  inflating: ./faces/68.jpg          
  inflating: ./faces/15587.jpg       
  inflating: ./faces/19473.jpg       
  inflating: ./faces/5854.jpg        
  inflating: ./faces/55116.jpg       
  inflating: ./faces/58435.jpg       
  inflating: ./faces/61469.jpg       
  inflating: ./faces/30281.jpg       
  inflating: ./faces/5654.jpg

In [3]:
!pip install stylegan2_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 5.0 MB/s eta 0:00

In [4]:
import math
import copy
from pathlib import Path
from random import random
from functools import partial
from collections import namedtuple
from multiprocessing import cpu_count

import numpy as np

import torch
from torch import nn, einsum
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torch.optim import Adam

import torchvision
from torchvision import transforms as T, utils

from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange

from PIL import Image
from tqdm.auto import tqdm
from ema_pytorch import EMA

from accelerate import Accelerator
import matplotlib.pyplot as plt
import os

torch.backends.cudnn.benchmark = True
torch.manual_seed(4096)

if torch.cuda.is_available():
  torch.cuda.manual_seed(4096)

## Step 1: Forward process (Noise scheduler)




In [5]:
def linear_beta_schedule(timesteps):
    """
    linear schedule, proposed in original ddpm paper
    """
    scale = 1000 / timesteps
    beta_start = scale * 0.0001
    beta_end = scale * 0.02
    return torch.linspace(beta_start, beta_end, timesteps, dtype = torch.float64)

def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

Create dataset

In [6]:
class Dataset(Dataset):
    def __init__(
        self,
        folder,
        image_size
    ):
        self.folder = folder
        self.image_size = image_size
        self.paths = [p for p in Path(f'{folder}').glob(f'**/*.jpg')]
        #################################
        ## TODO: Data Augmentation ##
        #################################
        self.transform = T.Compose([
            T.Resize(image_size),
            T.RandomHorizontalFlip(p=0.5),
            T.RandomVerticalFlip(p=0.5),
            T.RandomGrayscale(p=0.2),
            T.RandomRotation(90, interpolation=T.InterpolationMode.BILINEAR),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        img = Image.open(path)
        return self.transform(img)

# StyleGAN

In [ ]:
!stylegan2_pytorch --data ./faces --image-size 64 --batch-size 16 --num-train-steps 15000

2023-09-11 12:12:39.583666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
default<./faces>:   0% 50/15000 [01:35<7:55:10,  1.91s/it]G: 2.45 | D: 0.80 | GP: 0.38
default<./faces>:   1% 100/15000 [03:02<7:33:55,  1.83s/it]G: 0.17 | D: 1.46 | GP: 0.13
default<./faces>:   1% 150/15000 [04:32<7:29:03,  1.81s/it]G: -0.68 | D: 6.60 | GP: 1.31
default<./faces>:   1% 200/15000 [05:59<7:23:56,  1.80s/it]G: 1.43 | D: 1.75 | GP: 0.22
default<./faces>:   2% 250/15000 [07:29<7:21:43,  1.80s/it]G: 2.22 | D: 2.00 | GP: 0.73
default<./faces>:   2% 300/15000 [08:56<7:18:25,  1.79s/it]G: 0.24 | D: 1.17 | GP: 27.75
default<./faces>:   2% 350/15000 [10:25<7:16:42,  1.79s/it]G: 0.94 | D: 1.39 | GP: 0.03
default<./faces>:   3% 400/15000 [11:53<7:13:57,  1.78s/it]G: 0.05 | D: 0.74 | GP: 0.30
default<./faces>:   3% 450/15000 [13:22<7:12:25,  1.78s/it]G: 1.21 | D: 0.66 | GP: 3.05
default<./faces>:   3% 500/15000 [14:49<7:10:07,  1.78s/it]G: 1.28 | D: 0.75 | GP: 

In [ ]:
!stylegan2_pytorch --generate